# Pruebas de Hipótesis Práctica
Katlyn Goeujon-Mackness <br>
19/03/2025

## Estructura
1. Definir vectores y matrices para regresión lineal múltiple en Python
2. Calcular coeficientes de regresión y evaluar su significancia
3. Validar el modelo usando métricas como R² y R² ajustado
4. Identificar variables significativas usando pruebas de hipótesis en regresión
5. Analizar la multicolinealidad en modelos de regresión lineal múltiple
6. Evaluar la significancia de variables mediante el método Stepwise

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

Nota: Para este ejemplo asumiremos que los datos de entrenamiento son sobre los que trabajaremos.

### 1. Definir vectores y matrices para regresión lineal múltiple en Python

In [3]:
# Crear un DataFrame con datos dados
Y = np.array([[3], [1], [8], [3], [5]])
Y

array([[3],
       [1],
       [8],
       [3],
       [5]])

In [4]:
# Crear una matriz de características
X = np.array([[1, 3, 5], [1, 1, 4], [1, 5, 6], [1, 2, 4], [1, 4, 6]])
X

array([[1, 3, 5],
       [1, 1, 4],
       [1, 5, 6],
       [1, 2, 4],
       [1, 4, 6]])

In [5]:
# Calcular la transpuesta de X
XT_X = np.matmul(np.matrix.transpose(X), X)
XT_X

array([[  5,  15,  25],
       [ 15,  55,  81],
       [ 25,  81, 129]])

In [6]:
# Calcular la inversa de la matriz XTX_X
XT_X_inv = np.linalg.inv(XT_X)
XT_X_inv

array([[26.7,  4.5, -8. ],
       [ 4.5,  1. , -1.5],
       [-8. , -1.5,  2.5]])

In [7]:
# Calcular el producto de la transpuesta de X y Y
XT_Y = np.matmul(np.matrix.transpose(X), Y)
XT_Y

array([[ 20],
       [ 76],
       [109]])

### 2. Calcular coeficientes de regresión y evaluar su significancia


In [8]:
# Calcular los coeficientes de regresión (betas)
betas = np.matmul(XT_X_inv, XT_Y)
betas

array([[ 4. ],
       [ 2.5],
       [-1.5]])

In [9]:
# Cálculos de los pronósticos para Y de acuerdo a los coeficientes de regresión
Y_pred = np.matmul(X, betas)
Y_pred

array([[4. ],
       [0.5],
       [7.5],
       [3. ],
       [5. ]])

In [10]:
# Cálculo de residuales
Resid = Y - Y_pred
Resid

array([[-1.00000000e+00],
       [ 5.00000000e-01],
       [ 5.00000000e-01],
       [-1.33226763e-13],
       [-1.64313008e-13]])

In [11]:
# Cálculo de la suma de residuales al cuadrado
RSS = float(np.matmul(np.matrix.transpose(Resid), Resid))
RSS

1.4999999999999991

In [12]:
# Cálculo de la suma total de cuadrados
TSS = float(np.matmul(np.matrix.transpose(Y), Y) - len(Y)*(Y.mean()**2))
TSS

28.0

### 3. Validar el modelo usando métricas como R² y R² ajustado


In [13]:
# Cálculo del coeficiente de determinación R^2
R_cuad = float(1-(RSS/TSS))
R_cuad

0.9464285714285715

In [14]:
# Cálculo del coeficiente de determinación ajustado R^2
RSqAj = float(1 - (RSS / (X.shape[0] - X.shape[1])) / (TSS / (X.shape[0] - 1)))
RSqAj

0.8928571428571429

In [15]:
# Cálculo de la varianza del error de regresión
s_cuad = RSS / (len(Y) - X.shape[1])
s_cuad

0.7499999999999996

In [16]:
# Desviación estándar de los errores de regresión
import math
s = math.sqrt(s_cuad)
s

0.8660254037844384

In [17]:
# Cálculo de las t-stats para cada coeficiente de regresión
result_t = []
for i in range(0, X.shape[1]):
    t = float(betas[i] / (s * math.sqrt(XT_X_inv[i][i])))
    result_t.append(t)
result_t

[0.893868697538675, 2.8867513459481344, -1.0954451150103264]

### 4. Identificar variables significativas usando pruebas de hipótesis en regresión

### Criterio 1:

In [18]:
# Obtener valor crítico de la t de Student de tablas
import scipy.stats

grados_libertad = len(Y) - X.shape[1]
# La t_critica se obtendrá a un nivel de confianza del 95% (Alfa = 5%)
t_critico = abs(scipy.stats.t.ppf(q=0.025, df = grados_libertad))
t_critico

np.float64(4.302652729696144)

In [20]:
for i in range(0, X.shape[1]):
    if (abs(result_t[i])) > t_critico:
        print("Beta", i, "es significativa") # Aquí se rechaza H0
    else:
        print("Beta", i, "NO es significativa") # Aquí NO se rechaza H0

Beta 0 NO es significativa
Beta 1 NO es significativa
Beta 2 NO es significativa


Este resultado parece una contradicción. <br>
Continuamos con el siguente criterio.

### Criterio 2:

In [21]:
# Cálculo de valores p
for i in range(0, X.shape[1]):
    print("Valor p de Beta", i, ":", scipy.stats.t.sf(abs(result_t[i]), df = grados_libertad))

Valor p de Beta 0 : 0.23285799130426266
Valor p de Beta 1 : 0.05098674493306258
Valor p de Beta 2 : 0.19381378215210376


Si manejamos un nivel alfa de 5%, en ninguno de los casos el valor p es menor al 5%.
<br> Por lo que no podemos rechazar H0.

### Criterio 3

In [22]:
# Cálculo de intervales de confianze del 95% para el verdadero valor
#  del coeficiente de cada Beta
for i in range(0, X.shape[1]):
    print("El valor de Beta", i, "se encuentra entre", 
          float(betas[i]) - t_critico * s * math.sqrt(XT_X_inv[i][i]),
          "y", t_critico * s * math.sqrt(XT_X_inv[i][i]))
    

El valor de Beta 0 se encuentra entre -15.25407049846963 y 19.25407049846972
El valor de Beta 1 se encuentra entre -1.226206567579316 y 3.726206567579322
El valor de Beta 2 se encuentra entre -7.39164989291438 y 5.891649892914388


Ninguna de las variables regresoras (independientes) es significativamente distinta de cero.

In [23]:
# Comparación de resultados contra reporte automatizado
import statsmodels.api as sm

regressor = sm.OLS(Y, X).fit()
print(regressor.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.946
Model:                            OLS   Adj. R-squared:                  0.893
Method:                 Least Squares   F-statistic:                     17.67
Date:                Thu, 20 Mar 2025   Prob (F-statistic):             0.0536
Time:                        15:59:08   Log-Likelihood:                -4.0848
No. Observations:                   5   AIC:                             14.17
Df Residuals:                       2   BIC:                             13.00
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.0000      4.475      0.894      0.4

/home/codespace/.python/current/lib/python3.12/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


#### Comentario:
Los resultados de los criterios contradicen el R cuadrado y el R cuadrado ajustado, lo que sugiere sobreajuste. 
<br>La multicolinealidad podría ser el problema.

### 5. Analizar la multicolinealidad en modelos de regresión lineal múltiple


In [24]:
# Matriz de corelación
data = pd.DataFrame(X)
data2 = data.iloc[:,1:3]
data2.corr()

,1,2
1,1.000000,0.948683
2,0.948683,1.000000


La matriz de corelación sugiere multicolinealidad. Será necesario evaluar las variables.

### 6. Evaluar la significancia de variables mediante el método Stepwise

In [ ]:
# Eliminar la columna 2 de la matriz X
X_Nueva = np.delete(X, 2, 1)
X_Nueva

array([[1, 3],
       [1, 1],
       [1, 5],
       [1, 2],
       [1, 4]])

In [26]:
regressor = sm.OLS(Y, X_Nueva).fit()
print(regressor.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.886
Method:                 Least Squares   F-statistic:                     32.00
Date:                Thu, 20 Mar 2025   Prob (F-statistic):             0.0109
Time:                        16:08:07   Log-Likelihood:                -5.2598
No. Observations:                   5   AIC:                             14.52
Df Residuals:                       3   BIC:                             13.74
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8000      0.938     -0.853      0.4

/home/codespace/.python/current/lib/python3.12/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


#### Comentario
Al eliminar la columna de la matriz, apareció una variable que parece ser significativa.